In [25]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import random

%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
plt.style.use('seaborn')

import plotly.io as pio
import plotly.graph_objects as go

pio.templates.default = 'plotly'

pd.options.display.max_columns = 100
pd.options.display.max_rows = 400
import warnings
warnings.simplefilter('ignore')

from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import utils
import tstests

from tqdm import tqdm

In [81]:
temporal_features = ['dayofweek', 'month', 'weekofyear', 'year']

def add_temporal_features(df):
    df['dayofweek'] = df.index.dayofweek
    df['month'] = df.index.month
    df['weekofyear'] = df.index.isocalendar().week
    df['year'] = df.index.year
    
    return df

def get_prev_total(prev_df, df, curr_y, prev_y, feature='dayofweek'):
    grouped_df = prev_df.groupby([feature]).mean().sales
    grouped_df = grouped_df.reset_index()

    if feature == 'year':
        df.loc[
            (df.year==curr_y),
            f'lag1y_total_{feature}_mean'
        ] = grouped_df.sales.iloc[0]
    else:
        grouped_df['scaled'] = 0.1 + ((grouped_df.sales-grouped_df.sales.min())*(0.9-0.1))/(grouped_df.sales.max()-grouped_df.sales.min())
        for i, row in grouped_df.iterrows():
            df.loc[
                (df.year==curr_y) & (df[feature]==row[feature]),
                f'lag1y_total_{feature}_scaled'
            ] = row.scaled

def get_prev_si(prev_df, df, curr_y, prev_y, si, feature='dayofweek'):
    grouped_df = prev_df.groupby([si, feature]).mean().sales
    grouped_df = grouped_df.reset_index(level=[0,1])
    
    grouped_df_std = prev_df.groupby([si, feature]).std().sales
    grouped_df_std = grouped_df_std.reset_index(level=[0,1])
    
    if feature == 'year':
        for i, row in grouped_df.iterrows():
            df.loc[
                (df.year==curr_y) & (df[si]==row[si]),
                f'lag1y_{si}_{feature}_mean'
            ] = row.sales
    else:
        for i, row in grouped_df.iterrows():
            df.loc[
                (df.year==curr_y) & (df[si]==row[si]) & (df[feature]==row[feature]),
                f'lag1y_{si}_{feature}_mean'
            ] = row.sales
            
        for i, row in grouped_df_std.iterrows():    
            df.loc[
                (df.year==curr_y) & (df[si]==row[si]) & (df[feature]==row[feature]),
                f'lag1y_{si}_{feature}_std'
            ] = row.sales
    
def add_total_lag_sales(df):
    for i in range(1, len(df.year.unique())):
        curr_y = df.year.unique()[i]
        prev_y = df.year.unique()[i-1]
        prev_df = df.loc[df.year==prev_y]
        for t in temporal_features:
            get_prev_total(prev_df, df, curr_y, prev_y, feature=t)
            get_prev_si(prev_df, df, curr_y, prev_y, 'store', feature=t)
            get_prev_si(prev_df, df, curr_y, prev_y, 'item', feature=t)
            
    return df

def add_scaled_mean(df):
    for i in range(len(temporal_features)-1):
        tf = temporal_features[i]
        df[f'lag1y_store_{tf}_scaledmean'] = df[f'lag1y_total_{tf}_scaled'] * df[f'lag1y_store_{tf}_mean']
        df[f'lag1y_item_{tf}_scaledmean'] = df[f'lag1y_total_{tf}_scaled'] * df[f'lag1y_item_{tf}_mean']
        
    return df

def add_rolling_means(df):
    for lag in [90, 182, 365]:
        lag_col_name = f"lag_{lag}d"
        df[lag_col_name] = df.groupby(['item', 'store'])['sales'].shift(periods=lag)

    df['rolling_mean_90'] = df.groupby(['item', 'store'])['sales'].transform(lambda x: x.rolling(window=90).mean())
    df['rolling_mean_182'] = df.groupby(['item', 'store'])['sales'].transform(lambda x: x.rolling(window=182).mean())
    df['rolling_mean_365'] = df.groupby(['item', 'store'])['sales'].transform(lambda x: x.rolling(window=365).mean())
    df['rolling_std_90'] = df.groupby(['item', 'store'])['sales'].transform(lambda x: x.rolling(window=90).std())
    df['rolling_std_182'] = df.groupby(['item', 'store'])['sales'].transform(lambda x: x.rolling(window=182).std())
    df['rolling_std_365'] = df.groupby(['item', 'store'])['sales'].transform(lambda x: x.rolling(window=365).std())

    return df

def feature_engineer(df):
    df['date'] = df['date'].astype('datetime64[ns]')
    df = df.set_index('date')
    df = add_temporal_features(df)
    df = add_total_lag_sales(df)
    df = add_scaled_mean(df)
    df = add_rolling_means(df)
    
    return df

In [82]:
train_data = pd.read_csv("demand-forecasting-kernels-only/train.csv")
test_data = pd.read_csv("demand-forecasting-kernels-only/test.csv")

In [83]:
all_data = pd.concat([train_data, test_data])
all_data = feature_engineer(all_data)

In [84]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 958000 entries, 2013-01-01 to 2018-03-31
Data columns (total 41 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   store                              958000 non-null  int64  
 1   item                               958000 non-null  int64  
 2   sales                              913000 non-null  float64
 3   id                                 45000 non-null   float64
 4   dayofweek                          958000 non-null  int64  
 5   month                              958000 non-null  int64  
 6   weekofyear                         958000 non-null  UInt32 
 7   year                               958000 non-null  int64  
 8   lag1y_total_dayofweek_scaled       775500 non-null  float64
 9   lag1y_store_dayofweek_mean         775500 non-null  float64
 10  lag1y_store_dayofweek_std          775500 non-null  float64
 11  lag1y_item_dayofweek_me

In [85]:
all_data

,store,item,sales,id,dayofweek,month,weekofyear,year,lag1y_total_dayofweek_scaled,lag1y_store_dayofweek_mean,lag1y_store_dayofweek_std,lag1y_item_dayofweek_mean,lag1y_item_dayofweek_std,lag1y_total_month_scaled,lag1y_store_month_mean,lag1y_store_month_std,lag1y_item_month_mean,lag1y_item_month_std,lag1y_total_weekofyear_scaled,lag1y_store_weekofyear_mean,lag1y_store_weekofyear_std,lag1y_item_weekofyear_mean,lag1y_item_weekofyear_std,lag1y_total_year_mean,lag1y_store_year_mean,lag1y_item_year_mean,lag1y_store_dayofweek_scaledmean,lag1y_item_dayofweek_scaledmean,lag1y_store_month_scaledmean,lag1y_item_month_scaledmean,lag1y_store_weekofyear_scaledmean,lag1y_item_weekofyear_scaledmean,lag_90d,lag_182d,lag_365d,rolling_mean_90,rolling_mean_182,rolling_mean_365,rolling_std_90,rolling_std_182,rolling_std_365
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1,1,13.0,NaN,1,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02,1,1,11.0,NaN,2,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,1,1,14.0,NaN,3,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,1,1,13.0,NaN,4,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-05,1,1,10.0,NaN,5,1,1,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-27,10,50,NaN,44995.0,1,3,13,2018,0.366224,60.716923,28.474377,67.063462,19.783262,0.396504,59.780645,26.742627,67.632258,18.035212,0.463189,63.505714,29.452282,71.628571,20.52229,58.815014,66.010082,73.95589,22.235967,24.560219,23.703242,26.816435,29.415173,33.177595,63.0,79.0,66.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-28,10,50,NaN,44996.0,2,3,13,2018,0.379047,61.103846,28.427777,68.550000,20.009762,0.396504,59.780645,26.742627,67.632258,18.035212,0.463189,63.505714,29.452282,71.628571,20.52229,58.815014,66.010082,73.95589,23.161248,25.983692,23.703242,26.816435,29.415173,33.177595,59.0,80.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-29,10,50,NaN,44997.0,3,3,13,2018,0.515661,65.692692,30.644202,74.136538,21.819513,0.396504,59.780645,26.742627,67.632258,18.035212,0.463189,63.505714,29.452282,71.628571,20.52229,58.815014,66.010082,73.95589,33.875138,38.229298,23.703242,26.816435,29.415173,33.177595,74.0,82.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
split_date = pd.to_datetime(test_data.date[0])
train_df = all_data[all_data.index < split_date]
test_df = all_data[all_data.index >= split_date]
print(train_df.shape)
print(test_df.shape)

(913000, 41)
(45000, 41)


In [87]:
train_df = train_df.drop('id', axis=1)
train_df = train_df.dropna()

In [88]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 728500 entries, 2014-01-01 to 2017-12-31
Data columns (total 40 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   store                              728500 non-null  int64  
 1   item                               728500 non-null  int64  
 2   sales                              728500 non-null  float64
 3   dayofweek                          728500 non-null  int64  
 4   month                              728500 non-null  int64  
 5   weekofyear                         728500 non-null  UInt32 
 6   year                               728500 non-null  int64  
 7   lag1y_total_dayofweek_scaled       728500 non-null  float64
 8   lag1y_store_dayofweek_mean         728500 non-null  float64
 9   lag1y_store_dayofweek_std          728500 non-null  float64
 10  lag1y_item_dayofweek_mean          728500 non-null  float64
 11  lag1y_item_dayofweek_st

In [89]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 45000 entries, 2018-01-01 to 2018-03-31
Data columns (total 41 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   store                              45000 non-null  int64  
 1   item                               45000 non-null  int64  
 2   sales                              0 non-null      float64
 3   id                                 45000 non-null  float64
 4   dayofweek                          45000 non-null  int64  
 5   month                              45000 non-null  int64  
 6   weekofyear                         45000 non-null  UInt32 
 7   year                               45000 non-null  int64  
 8   lag1y_total_dayofweek_scaled       45000 non-null  float64
 9   lag1y_store_dayofweek_mean         45000 non-null  float64
 10  lag1y_store_dayofweek_std          45000 non-null  float64
 11  lag1y_item_dayofweek_mean          45

# XGB

In [90]:
import xgboost as xgb

In [91]:
xgb_if = ['rolling_mean_90',
 'rolling_mean_365',
 'lag1y_item_dayofweek_scaledmean',
 'lag1y_item_month_mean',
 'lag1y_store_dayofweek_scaledmean',
 'lag1y_item_month_scaledmean',
 'lag1y_total_month_scaled',
 'rolling_std_182',
 'lag1y_store_month_scaledmean',
 'lag1y_store_month_std',
 'lag1y_store_dayofweek_mean',
 'lag_365d',
 'lag1y_item_month_std',
 'rolling_std_90',
 'lag1y_store_month_mean',
 'month',
 'rolling_mean_182']

rf_if = ['rolling_mean_90',
 'lag_365d',
 'rolling_std_182',
 'rolling_mean_365',
 'lag1y_item_dayofweek_scaledmean',
 'lag1y_store_dayofweek_scaledmean',
 'lag1y_item_month_mean',
 'lag1y_item_month_scaledmean',
 'rolling_std_365',
 'lag1y_store_month_scaledmean',
 'lag1y_total_dayofweek_scaled',
 'lag1y_total_month_scaled',
 'dayofweek',
 'lag1y_store_dayofweek_mean',
 'rolling_std_90',
 'lag1y_store_dayofweek_std',
 'lag1y_item_month_std',
 'lag1y_store_month_mean',
 'lag1y_store_month_std',
 'lag1y_item_weekofyear_scaledmean']

In [92]:
[i for i in xgb_if if i not in rf_if]

['month', 'rolling_mean_182']

In [93]:
[i for i in rf_if if i not in xgb_if]

['rolling_std_365',
 'lag1y_total_dayofweek_scaled',
 'dayofweek',
 'lag1y_store_dayofweek_std',
 'lag1y_item_weekofyear_scaledmean']

In [94]:
selected_features = rf_if+['month', 'rolling_mean_182']

In [95]:
train_df[['month', 'rolling_mean_182']]

,month,rolling_mean_182
date,,
2014-01-01,1,17.445055
2014-01-02,1,17.390110
2014-01-03,1,17.357143
2014-01-04,1,17.379121
2014-01-05,1,17.285714
...,...,...
2017-12-27,12,86.752747
2017-12-28,12,86.406593
2017-12-29,12,86.197802


In [96]:
X_train = train_df[selected_features]
X_test = test_df[selected_features]
y_train = train_df.sales

booster_params = {
        'booster' : 'gbtree', 
        'n_estimators' : 5000,
        'early_stopping_rounds' : 50,
        'objective' : 'reg:squarederror',
        'max_depth' : 3,
        'learning_rate' : 0.01
}

xgbreg = xgb.XGBRegressor(**booster_params)
xgbreg.fit(X_train,
        y_train,
        eval_set=[(X_train, y_train)],
        verbose=100)
train_df['xgb'] = xgbreg.predict(X_train)

[0]	validation_0-rmse:29.33140
[100]	validation_0-rmse:14.96510
[200]	validation_0-rmse:10.20515
[300]	validation_0-rmse:8.74210
[400]	validation_0-rmse:8.23726
[500]	validation_0-rmse:8.03719
[600]	validation_0-rmse:7.94077
[700]	validation_0-rmse:7.87332
[800]	validation_0-rmse:7.82396
[900]	validation_0-rmse:7.78780
[1000]	validation_0-rmse:7.75868
[1100]	validation_0-rmse:7.72980
[1200]	validation_0-rmse:7.70394
[1300]	validation_0-rmse:7.68151
[1400]	validation_0-rmse:7.66239
[1500]	validation_0-rmse:7.64543
[1600]	validation_0-rmse:7.63018
[1700]	validation_0-rmse:7.61652
[1800]	validation_0-rmse:7.60412
[1900]	validation_0-rmse:7.59260
[2000]	validation_0-rmse:7.58219
[2100]	validation_0-rmse:7.57243
[2200]	validation_0-rmse:7.56324
[2300]	validation_0-rmse:7.55508
[2400]	validation_0-rmse:7.54728
[2500]	validation_0-rmse:7.53990
[2600]	validation_0-rmse:7.53286
[2700]	validation_0-rmse:7.52630
[2800]	validation_0-rmse:7.52021
[2900]	validation_0-rmse:7.51450
[3000]	validation_0

In [97]:
utils.sMAPE(train_df.sales, train_df.xgb)

12.305761628187106

In [98]:
test_df['sales'] = xgbreg.predict(X_test)

In [99]:
test_df

,store,item,sales,id,dayofweek,month,weekofyear,year,lag1y_total_dayofweek_scaled,lag1y_store_dayofweek_mean,lag1y_store_dayofweek_std,lag1y_item_dayofweek_mean,lag1y_item_dayofweek_std,lag1y_total_month_scaled,lag1y_store_month_mean,lag1y_store_month_std,lag1y_item_month_mean,lag1y_item_month_std,lag1y_total_weekofyear_scaled,lag1y_store_weekofyear_mean,lag1y_store_weekofyear_std,lag1y_item_weekofyear_mean,lag1y_item_weekofyear_std,lag1y_total_year_mean,lag1y_store_year_mean,lag1y_item_year_mean,lag1y_store_dayofweek_scaledmean,lag1y_item_dayofweek_scaledmean,lag1y_store_month_scaledmean,lag1y_item_month_scaledmean,lag1y_store_weekofyear_scaledmean,lag1y_item_weekofyear_scaledmean,lag_90d,lag_182d,lag_365d,rolling_mean_90,rolling_mean_182,rolling_mean_365,rolling_std_90,rolling_std_182,rolling_std_365
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-01,1,1,65.472794,0.0,0,1,1,2018,0.100000,42.221538,20.423021,19.615385,7.020624,0.100000,36.184516,16.691157,16.890323,5.508531,0.104923,36.477143,17.083087,16.728571,5.550614,58.815014,53.247890,24.699452,4.222154,1.961538,3.618452,1.689032,3.827293,1.755213,18.0,18.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-02,1,1,68.565109,1.0,1,1,1,2018,0.366224,49.082692,23.581571,22.800000,8.074699,0.100000,36.184516,16.691157,16.890323,5.508531,0.104923,36.477143,17.083087,16.728571,5.550614,58.815014,53.247890,24.699452,17.975238,8.349897,3.618452,1.689032,3.827293,1.755213,15.0,23.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,1,1,67.497719,2.0,2,1,1,2018,0.379047,49.350385,23.164382,23.051923,7.648038,0.100000,36.184516,16.691157,16.890323,5.508531,0.104923,36.477143,17.083087,16.728571,5.550614,58.815014,53.247890,24.699452,18.706130,8.737769,3.618452,1.689032,3.827293,1.755213,20.0,23.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-04,1,1,73.062019,3.0,3,1,1,2018,0.515661,52.803462,24.784664,24.513462,8.213189,0.100000,36.184516,16.691157,16.890323,5.508531,0.104923,36.477143,17.083087,16.728571,5.550614,58.815014,53.247890,24.699452,27.228669,12.640628,3.618452,1.689032,3.827293,1.755213,19.0,31.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05,1,1,73.682205,4.0,4,1,1,2018,0.636830,56.127692,26.581328,26.038462,8.680724,0.100000,36.184516,16.691157,16.890323,5.508531,0.104923,36.477143,17.083087,16.728571,5.550614,58.815014,53.247890,24.699452,35.743817,16.582082,3.618452,1.689032,3.827293,1.755213,22.0,32.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-03-27,10,50,102.688347,44995.0,1,3,13,2018,0.366224,60.716923,28.474377,67.063462,19.783262,0.396504,59.780645,26.742627,67.632258,18.035212,0.463189,63.505714,29.452282,71.628571,20.522290,58.815014,66.010082,73.955890,22.235967,24.560219,23.703242,26.816435,29.415173,33.177595,63.0,79.0,66.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-28,10,50,101.711273,44996.0,2,3,13,2018,0.379047,61.103846,28.427777,68.550000,20.009762,0.396504,59.780645,26.742627,67.632258,18.035212,0.463189,63.505714,29.452282,71.628571,20.522290,58.815014,66.010082,73.955890,23.161248,25.983692,23.703242,26.816435,29.415173,33.177595,59.0,80.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-29,10,50,113.996307,44997.0,3,3,13,2018,0.515661,65.692692,30.644202,74.136538,21.819513,0.396504,59.780645,26.742627,67.632258,18.035212,0.463189,63.505714,29.452282,71.628571,20.522290,58.815014,66.010082,73.955890,33.875138,38.229298,23.703242,26.816435,29.415173,33.177595,74.0,82.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
final_df = test_df[['id', 'sales']]
final_df.id = final_df.id.astype(int)
final_df.head()

,id,sales
date,,
2018-01-01,0,65.472794
2018-01-02,1,68.565109
2018-01-03,2,67.497719
2018-01-04,3,73.062019
2018-01-05,4,73.682205


In [101]:
final_df.to_csv('submissions/submission_xgb_selected_features.csv', index=False)